# Task3 基于机器学习的文本分类
本次学习使用机器学习模型来解决文本分类。机器学习发展比较广，且包括多个分支，本章侧重使用传统机器学习，从下一章开始是基于深度学习的文本分类。
### 学习目标
- 学会TF-IDF的原理和使用
- 使用sklearn的机器学习模型完成文本分类  

### 机器学习模型
机器学习是对能通过经验自动改进的计算机算法的研究。机器学习通过**历史数据**训练出模型对应于人类对**经验**进行**归纳**的过程，机器学习利用模型对**新数据**进行预测对应于人类利用总结的**规律**对新问题进行**预测**的过程。 

机器学习有很多种分支，对于学习者来说应该优先掌握机器学习算法的分类，然后再其中一种机器学习算法进行学习。  
对于机器学习初学者，应该知道以下事情：  
1. 机器学习能解决一定的问题，但不能奢求机器学习是万能的；
2. 机器学习算法有很多种，看具体问题需要什么，再来进行选择；
3. 每种机器学习算法有一定的偏好，需要具体问题具体分析；
![](https://camo.githubusercontent.com/d8e9a12417a2a2a754a874af0ae163bb1bddbb0b/68747470733a2f2f696d672d626c6f672e6373646e696d672e636e2f32303230303731343230333232333235332e6a7067)  

## 文本表示方法 Part1
在机器学习算法的训练过程中，假设给定$N$个样本，每个样本有$M$个特征，这样组成了$N×M$的样本矩阵，然后完成算法的训练和预测。同样的在计算机视觉中可以将图片的像素看作特征，每张图片看作hight×width×3的特征图，一个三维的矩阵来进入计算机进行计算。  

但是在自然语言领域，上述方法却不可行：文本是不定长度的。文本表示成计算机能够运算的数字或向量的方法一般称为词嵌入（Word Embedding）方法。词嵌入将不定长的文本转换到定长的空间内，是文本分类的第一步。 
### 1.什么是Embedding？
Embedding在数学上表示一个映射关系，F: X -> Y， 也就是一个Function，其中该函数是injective（就是我们所说的单射函数，每个Y只有唯一的X对应，反之亦然）和structure-preserving (结构保存，比如在X所属的空间上X1 < X2,那么映射后在Y所属空间上同理 Y1 < Y2)。那么对于word embedding，就是将单词word映射到另外一个空间，其中这个映射具有injective和structure-preserving的特点。  

深度学习中，Embedding 特指用一个低维度向量表示一个实体，实体可以是一个词(Word2Vec)，可以是一个物品(Item2Vec)，亦或者网络关系中的节点(Graph Embedding)。

举个栗子：
使用Word2Vec将单词映射到新的向量空间，获得单词的新的表达。从图中可以得出：
Embedding(Moscow) - Embedding(Russia) ≈ Embedding(Tokyo) - Embedding(Japan)，即 Embedding 之后向量可以进行计算。并且，距离相近的向量对应的实体有相近的含义。
![](https://img-blog.csdnimg.cn/20200725211526406.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80NDE1NDM5Mw==,size_16,color_FFFFFF,t_70,=50*20)
### 2.One-hot
在回归，分类，聚类等机器学习算法中，特征之间距离的计算或相似度的计算是非常重要的，而我们常用的距离或相似度的计算都是在欧式空间的相似度计算，计算余弦相似性，基于的就是欧式空间。

而我们使用one-hot编码，将离散特征的取值扩展到了欧式空间，离散特征的某个取值就对应欧式空间的某个点。

将离散型特征使用one-hot编码，确实会让特征之间的距离计算更加合理。

比如，有一个离散型特征，代表工作类型，该离散型特征，共有三个取值，不使用one-hot编码，其表示分别是x_1 = (1), x_2 = (2), x_3 = (3)。两个工作之间的距离是，(x_1, x_2) = 1, d(x_2, x_3) = 1, d(x_1, x_3) = 2。那么x_1和x_3工作之间就越不相似吗？显然这样的表示，计算出来的特征的距离是不合理。那如果使用one-hot编码，则得到x_1 = (1, 0, 0), x_2 = (0, 1, 0), x_3 = (0, 0, 1)，那么两个工作之间的距离就都是sqrt(2).即每两个工作之间的距离是一样的，显得更合理。

这里的One-hot与数据挖掘任务中的操作是一致的，即将每一个单词使用一个离散的向量表示。具体将每个字/词编码一个索引，然后根据索引进行赋值。  

One-hot表示方法的例子如下：  
```
句子1：我 爱 北 京 天 安 门
句子2：我 喜 欢 上 海
```
首先对所有句子的字进行索引，即将每个字确定一个编号：  
```
{'我': 1, '爱': 2, '北': 3, '京': 4, '天': 5,
  '安': 6, '门': 7, '喜': 8, '欢': 9, '上': 10, '海': 11
}
```
在这里共包括11个字，因此每个字可以转换为一个11维度稀疏向量：  
```
我：[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
爱：[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
...
海：[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
```

这样每个词的取值就可以拓展到向量空间中的某一个点了，通过计算两个词在向量空间的距离来研究他们之间的关系。但当单词的个数越来越多的时候，这个稀疏矩阵也会越来越大。  
### Bag of Words
Bag of Words（词袋），也称为Count Vectors，是信息检索领域常用的文档表示方法。每个文档的字/词可以使用其出现次数来进行表示。词频统计可以得知某个词在句子中出现的次数，但缺点是无法得知词与词之间的关系和词在原来句子中出现的次序。  

在信息检索中，BOW模型假定对于一个文档，忽略它的单词顺序和语法、句法等要素，将其仅仅看作是若干个词汇的集合，文档中每个单词的出现都是独立的，不依赖于其它单词是否出现。（是不关顺序的）。  
```
句子1：我 爱 北 京 天 安 门
句子2：我 喜 欢 上 海
```
直接统计每个字出现的次数，并进行赋值：  
```
句子1：我 爱 北 京 天 安 门
转换为 [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]

句子2：我 喜 欢 上 海
转换为 [1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
```
这两个向量共包含11个元素，其中第i个元素表示字典中第i个单词在句子中出现的次数。因此BOW模型可认为是一种统计直方图 (histogram)。在文本检索和处理应用中, 可以通过该模型很方便的计算词频。  
在sklearn中可以直接用`CountVectorizer`来实现这一步骤：  
```
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]
vectorizer = CountVectorizer()
vectorizer.fit_transform(corpus).toarray()
```
### N-gram
N-Gram是一种基于统计语言模型的算法。它的基本思想是将文本里面的内容按照字节进行大小为N的滑动窗口操作，形成了长度是N的字节片段序列。

每一个字节片段称为gram，对所有gram的出现频度进行统计，并且按照事先设定好的阈值进行过滤，形成关键gram列表，也就是这个文本的向量特征空间，列表中的每一种gram就是一个特征向量维度。  

该模型基于这样一种假设，第N个词的出现只与前面N-1个词相关，而与其它任何词都不相关，整句的概率就是各个词出现概率的乘积。这些概率可以通过直接从语料中统计N个词同时出现的次数得到。常用的是二元的Bi-Gram和三元的Tri-Gram。  

N-gram与Count Vectors类似，不过加入了相邻单词组合成为新的单词，并进行计数。  
如果N取值为2，则句子1和句子2就变为：  
```
句子1：我爱 爱北 北京 京天 天安 安门
句子2：我喜 喜欢 欢上 上海
```
这样一定程度上可以保持句子的连续性，但是一旦句子长了，向量会变得十分冗长。  
### TF-IDF
TF-IDF(term frequency–inverse document frequency)是一种用于信息检索与数据挖掘的常用加权技术，常用于挖掘文章中的关键词，而且算法简单高效，常被工业用于最开始的文本数据清洗。  

TF-IDF有两层意思，一层是"词频"（Term Frequency，缩写为TF），另一层是"逆文档频率"（Inverse Document Frequency，缩写为IDF）。其中计算语料库中文档总数除以含有该词语的文档数量，然后再取对数就是逆文档频率。

```
TF(t)= 该词语在当前文档出现的次数 / 当前文档中词语的总数
IDF(t)= log_e（文档总数 / 出现该词语的文档总数）
```
当有TF(词频)和IDF(逆文档频率)后，将这两个词相乘，就能得到一个词的TF-IDF的值。某个词在文章中的TF-IDF越大，那么一般而言这个词在这篇文章的重要性会越高，所以通过计算文章中各个词的TF-IDF，由大到小排序，排在最前面的几个词，就是该文章的关键词。

TF-IDF的优点是简单快速，而且容易理解。缺点是有时候用词频来衡量文章中的一个词的重要性不够全面，有时候重要的词出现的可能不够多，而且这种计算无法体现位置信息，无法体现词在上下文的重要性。如果要体现词的上下文结构，那么你可能需要使用word2vec算法来支持。 

TF_IDF可以权衡每个词对句子的重要性，但丢失了语序，只能表示词的出现与否。  
## 岭回归分类器（RidgeClassifier）
岭回归即在普通线性回归模型经验风险（损失函数）的基础上加入一个L2正则项，在使其最小化的过程中对模型参数进行约束，避免过拟合。该模型主要针对自变量之间存在多重共线性或者自变量个数多于样本量的情况。
## 基于机器学习的文本分类
接下来我们将对比不同文本表示算法的精度，通过本地构建验证集计算F1得分。  
### Count Vectors + RidgeClassifier

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import f1_score

train_df = pd.read_csv('train_set.csv', sep='\t', nrows=15000)

vectorizer = CountVectorizer(max_features=3000)
train_test = vectorizer.fit_transform(train_df['text'])

clf = RidgeClassifier()
clf.fit(train_test[:10000], train_df['label'].values[:10000])

val_pred = clf.predict(train_test[10000:])
print(f1_score(train_df['label'].values[10000:], val_pred, average='macro'))

0.65441877581244


### TF-IDF + RidgeClassifier
随机森林本质上是许多以不同方式过拟合的决策树的集合，该模型作为Bagging算法的一种，通过对这些决策树集合的结果取平均值来降低过拟合，以获得更优的预测效果。

In [2]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import f1_score

train_df = pd.read_csv('train_set.csv', sep='\t', nrows=15000)

tfidf = TfidfVectorizer(ngram_range=(1,3), max_features=3000)
train_test = tfidf.fit_transform(train_df['text'])

clf = RidgeClassifier()
clf.fit(train_test[:10000], train_df['label'].values[:10000])

val_pred = clf.predict(train_test[10000:])
print(f1_score(train_df['label'].values[10000:], val_pred, average='macro'))

0.8719372173702
